In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from collections import Counter
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tziam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv("Datasets/train.csv")
test_data = pd.read_csv("Datasets/test.csv")
sample_submission = pd.read_csv("Datasets/sample_submission.csv")

In [3]:
data.dropna(axis = 0, how ='any',inplace=True)

In [4]:
print(set(data['sentiment']))
y=data['selected_text']
X=data
X.head(n=10)

{'positive', 'negative', 'neutral'}


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.15)

In [6]:
def clean(text):
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text.lower()

In [7]:
# Returns a specific column from a DataFrame based on the specified sentiment.
def get_column_from_sentiment(sentiment,column):
    selected_data=X_train.loc[X_train['sentiment'] == sentiment]
    return selected_data[column]

# Calculates the word count and frequency of each word in a given column.
def get_word_count(column):
    all_text="".join(column)
    all_text=clean(all_text)
    all_text=all_text.split(' ')
    all_text = list(filter(None, all_text))
    total=len(all_text)
    counts = Counter(all_text)
    for item, count in counts.items():
        counts[item] /= total
    return counts

In [8]:
text=X_train['text']
selected_text=y_train

# Join the text and selected text into single strings
text="".join(text)
selected_text="".join(selected_text)

# Preprocess the text and selected text
text=clean(text)
selected_text=clean(selected_text)

# Split the text and selected text into individual words
text=text.split(' ')
selected_text=selected_text.split(' ')

# Remove any empty strings from the lists
text = list(filter(None, text))
selected_text = list(filter(None, selected_text))

# Calculate the word count and frequency in the text
text_counter = get_word_count(text)

# Find the words present in the text but not in the selected text
list_difference = [item for item in text if item not in selected_text]

In [9]:
# Converts a column in a DataFrame to a list of preprocessed and filtered words.
def column_to_list(column):
    text="".join(column)
    text=clean(text)
    text=text.split(' ')
    text = list(filter(None, text))
    return text

# Finds the difference between two columns by comparing their word lists.
def get_list_difference(column_mother,column_child):
    list_mother=column_to_list(column_mother)
    list_child=column_to_list(column_child)
    list_difference = [item for item in list_mother if item not in list_child]
    return list_difference

In [10]:
# Calculates the number of words in a sentence.
def get_length(sentence):
    words=sentence.split(' ')
    words = list(filter(None, words))
    return len(words)

# Calculates the average length of sentences in a column.
def average_length(column):
    length=[]
    for i in range(len(column)):
        sentence=column.iloc[i]
        length.append(get_length(sentence))
    print(np.max(length))
print(average_length(X_train['selected_text']))

33
None


In [11]:
# Get the word counts for each sentiment category
positive_counter=get_word_count(get_column_from_sentiment('positive', X_train['sentiment']))
negative_counter=get_word_count(get_column_from_sentiment('negative', X_train['sentiment']))
neutral_counter=get_word_count(get_column_from_sentiment('neutral', X_train['sentiment']))

# Get the lists of words for each sentiment category
positive_list=list(positive_counter.keys())
negative_list=list(negative_counter.keys())
neutral_list=list(neutral_counter.keys())

# Find the words that are common across all sentiment categories
common_elements = list(set(positive_list).intersection(set(negative_list)))
common_elements = list(set(common_elements).intersection(set(neutral_list)))

# Initialize an empty list to store the words that meet the specified conditions
no_words=[]
# neighbor is a threshold parameter for comparing word frequencies
neighbor=0

# Iterate over the common words and check the conditions using sentiment frequencies
for w in common_elements:
    if (positive_counter[w]> (1-neighbor)*negative_counter[w] and positive_counter[w]< (1+neighbor)*negative_counter[w]) and (neutral_counter[w]> (1-neighbor)*negative_counter[w] and neutral_counter[w]< (1+neighbor)*negative_counter[w]):
        no_words.append(w)
print(no_words)

KeyError: "None of [Index(['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',\n       'positive', 'negative', 'negative', 'negative',\n       ...\n       'positive', 'positive', 'neutral', 'positive', 'negative', 'neutral',\n       'positive', 'negative', 'neutral', 'neutral'],\n      dtype='object', length=23358)] are in the [columns]"

In [12]:
# Compares the word frequencies between a parent column and a child column.
def freq_comparison(column_parent,column_child):
    parent_count=get_word_count(column_parent)
    child_count=get_word_count(column_child)
    remove_list=[]
    for w in parent_count:
        if w not in child_count:
            remove_list.append(w)
        elif parent_count[w] > 4* child_count[w]:
            remove_list.append(w)
    return remove_list

positive=X_train.loc[X_train['sentiment'] == 'positive']
remove_list_positive=freq_comparison(positive['text'],positive['selected_text'])

neutral=X_train.loc[X_train['sentiment'] == 'neutral']
remove_list_neutral=freq_comparison(neutral['text'],neutral['selected_text'])

negative=X_train.loc[X_train['sentiment'] == 'negative']
remove_list_negative=freq_comparison(negative['text'],negative['selected_text'])

In [13]:
# Create a subset of the DataFrame containing only positive sentiment rows
positive=X_train.loc[X_train['sentiment'] == 'positive']
# Calculate the list of words that are present in the 'text' column but not in the 'selected_text' column of the positive subset
remove_list_positive=get_list_difference(positive['text'],positive['selected_text'])
print(len(set(remove_list_positive)))

1999


In [14]:
neutral=X_train.loc[X_train['sentiment'] == 'neutral']
remove_list_neutral=get_list_difference(neutral['text'],neutral['selected_text'])

In [15]:
negative=X_train.loc[X_train['sentiment'] == 'negative']
remove_list_negative=get_list_difference(negative['text'],negative['selected_text'])

In [16]:
print(len(remove_list_negative))

6695


In [17]:
# Predicts the selected words in a given sentence based on the specified sentiment.
def predict(sentence,sentiment):
    # Preprocess the input sentence using the `preprocess_tweet_text` function
    sentence=clean(sentence)
    # Convert the preprocessed sentence to lowercase and split into individual words
    words=sentence.lower().split()

    # Determine the appropriate remove_list based on the specified sentiment
    if sentiment=='positive':
        remove_list=remove_list_positive
    if sentiment=='neutral':
        remove_list=[]
    if sentiment=='negative':
        remove_list=remove_list_negative

    # Filter the words in the sentence by removing the ones present in the remove_list
    selected_words=[w for w in words if w not in remove_list]
    # Join the selected words back into a single string
    selected_words=' '.join(selected_words)

    return selected_words

In [18]:
# Calculates the Jaccard similarity coefficient between two strings.
def jaccard(str1, str2):
    # Convert the first string to lowercase and split it into individual words. Create a set from the words.
    a = set(str1.lower().split())
    # Convert the second string to lowercase and split it into individual words. Create a set from the words.
    b = set(str2.lower().split())
    # Find the intersection of the two sets of words.
    c = a.intersection(b)

    return float(len(c)) / (len(a) + len(b) - len(c))

In [19]:
# Initialize an empty list to store the Jaccard similarity coefficients
jac=[]

# Iterate over rows to make predictions
for i in range(X_test.shape[0]): 
    sentence=X_test['text'].iloc[i]
    sentiment=X_test['sentiment'].iloc[i]
    selected_sentence=y_test.iloc[i]
    if len(sentence.split())>5:
        prediction=predict(sentence,sentiment)
    else:
        prediction=sentence
    jac.append(jaccard(selected_sentence,prediction))

print(np.mean(jac))

0.5729263876589449


In [20]:
X_test.head()

,textID,text,selected_text,sentiment
1976,a8296482f1,@ the San Antonio Missions game waitressing. N...,@ the San Antonio Missions game waitressing. N...,neutral
14522,1d450fad81,Had an interesting evening amongst homeless pe...,interesting,positive
16048,3e3a0af796,"Try being in AZ again brooke, Omg you cant ev...","Try being in AZ again brooke, Omg you cant eve...",negative
16981,f3bd26a6ff,what happened? I missed 4 hours of updates,what happened? I missed 4 hours of updates,neutral
25085,3a70785fec,You cna get em for about ï¿½17 but I`ve got ...,You cna get em for about ï¿½17 but I`ve got on...,neutral


In [21]:
sample_submission=sample_submission.drop('selected_text',axis=1)
for i in range(test_data.shape[0]):
    sentence=test_data['text'].iloc[i]
    sentiment = test_data['sentiment'].iloc[i]
    if len(sentence.split())>2:
        prediction=predict(sentence, sentiment)
    else:
        prediction=sentence
    sample_submission.at[i,'selected_text']=prediction

In [22]:
sample_submission.to_csv('submission.csv',index=False)

In [23]:
sample_submission.head()

,textID,selected_text
0,f87dea47db,last session of the day
1,96d74cb729,shanghai is also really exciting (precisely --...
2,eee518ae67,"recession hit veronique branquinho, she has to..."
3,01082688c6,happy bday!
4,33987a8ee5,- i like it!!
